In [23]:
#from utils import progress_bar
import torch.optim as optim
#from utils import progress_bar
import torch.optim as optim
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [12]:
def get_cifar10():
    print('==> Preparing data..')
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    trainset = torchvision.datasets.CIFAR10(
        root='/media/jake/mark-4tb3/input', train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=128, shuffle=True, num_workers=2)

    testset = torchvision.datasets.CIFAR10(
        root='/media/jake/mark-4tb3/input', train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=128, shuffle=False, num_workers=2)

    classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

    return trainloader,testloader


In [13]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# torch dataset has map-style datasets, iterable-style dataset
trainloader,testloader = get_cifar10()
print(trainloader.dataset)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /media/jake/mark-4tb3/input
    Split: Train
    StandardTransform
Transform: Compose(
               RandomCrop(size=(32, 32), padding=4)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
           )


# Models

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # 32 x 32
        # input_channel(3), output_channel(6), filter(5)
        self.conv1 = nn.Conv2d(3,6,5) #in_channels,out_channels,kernel_size,stride,padding
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16 * 5 * 5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):


        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [26]:
net = Net()
net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001,momentum = 0.9)

# Train

In [25]:
def train(path):
    #global net
    #net = Resnet.resnet()
    net.cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr = 0.001,momentum = 0.9)

    for epoch in range(2):
        running_loss = 0
        for i,data in enumerate(trainloader,0):
            inputs, labels = data
            inputs, labels = inputs.cuda(),labels.cuda()
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 2000 == 1999:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
    print('Finished Training')

    torch.save(net.state_dict(),path)

def test(path):
    #net = resnet()
    net.cuda()
    net.load_state_dict(torch.load(path))

    dataiter = iter(testloader)
    images, labels = dataiter.next()
    images,labels = images.cuda(),labels.cuda()

    outputs = net(images)

    _,predicted = torch.max(outputs,1)

    print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                                  for j in range(4)))

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images,labels = images.cuda(),labels.cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
            100 * correct / total))
def main():
    path = '/media/jake/mark-4tb3/input/pytorch/cifar10/net.pth'
    for i in range(3):
        print('start train')
        train(path)
        print('start test')
        test(path)

if __name__ == '__main__':
    main()

start train
Finished Training
start test
Predicted:    cat  ship  ship  ship
Accuracy of the network on the 10000 test images: 36 %
start train
Finished Training
start test
Predicted:    cat  ship  ship  ship
Accuracy of the network on the 10000 test images: 42 %
start train
Finished Training
start test
Predicted:    cat  ship  ship  ship
Accuracy of the network on the 10000 test images: 46 %
